# Aufgabe-G-WiederholteFragen

## Allgemeines

Eine allgemeine Beschreibung der Laboraufgaben inklusive des Vorgehens, den Bewertungsrichtlinien und der Abgabe finden Sie  <a href="ML-allgemein.ipynb">hier</a>

## Datenquelle


* Laden Sie ihre Daten von http://141.72.190.207/ml_lab/G_fragen herunter
    * Die Daten sind geschützt. 
        * Sie müssen evtl. in einem Netzwerk der DHBW (z.B. WLAN, VPN, ...) angemeldet sein. 
        * Sie können sich auf der Webseite mit dem Benutzernamen dhbw und dem Zugangsnamen ml_2021 anmelden. 
* Die Daten sind in einem anwendungsspezifischen Format gespeichert.
    * Sie finden evtl. Informationen über die Daten in einer "README" Datei. 
    * Finden Sie keine solche Datei sind die Daten selbst erklärend. 
    
    



## Aufgabe

Die Datensammlung enthält Fragen aus einem Forum. Es sind Paare aus Fragen gleichen Inhalts geboten.

Erstellen Sie ein tiefes Neuronales Netzwerk, dass dem Nutzer schon bei der Eingabe einer neuen Frage gleich ähnliche Fragen anbietet. 
* Je mehr Text der Benutzer eingibt, desto präziser sollen die ähnlichen Fragen sein! 




# Lösung

* Beginnen Sie hier mit Ihrer Dokumentation und Implementierung! 

In [8]:
# Fügen Sie ihren Code hier ein!

Vorgehen:
- Finden von unnötigen Wörtern (in, wie usw.)
- Extrahieren der "wertvollen" Informationswerten
- Einteilen der Fragen in Kategorien und Lernen KI anhand der gelieferten Paare
- Überprüfung anhand der Testfragen
- Erstellung einer Eingabemaske, mit Testfragen zur überprüfung der KI

# Einlesen der Daten

## Einlesen der Trainingsdaten

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

train_data = pd.read_csv(
    "./fragen/train.csv",
    usecols= ['question1', 'question2', 'is_duplicate'],
    encoding='utf-8'
)

train_data.dropna(inplace=True)
train_data.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


## Einlesen der Testdaten

In [2]:
test_data = pd.read_csv(
    "./fragen/test.csv",
    index_col="test_id"
)

test_data.head()


/tmp/ipykernel_3550/1675913220.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(


,question1,question2
test_id,,
0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,What but is the best way to send money from Ch...,What you send money to China?
3,Which food not emulsifiers?,What foods fibre?
4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [3]:
char_set = set(train_data)
char_set

{'is_duplicate', 'question1', 'question2'}

# Text preprocessing
According to the tensorflow website

In [4]:
target = train_data.pop('is_duplicate')

ds_raw = tf.data.Dataset.from_tensor_slices((train_data.values, target.values))

for ex in ds_raw.take(3):
    print(ex)

(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'What is the step by step guide to invest in share market in india?',
       b'What is the step by step guide to invest in share market?'],
      dtype=object)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
       b'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?'],
      dtype=object)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'How can I increase the speed of my internet connection while using a VPN?',
       b'How can Internet speed be increased by hacking through DNS?'],
      dtype=object)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


2022-02-24 10:32:20.061404: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-24 10:32:20.162175: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-24 10:32:20.162357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-24 10:32:20.164385: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

## Unterteilung in Trainings, Validierungs und Testdaten


In [5]:
tf.random.set_seed(1)

ds_length = len(ds_raw)
ds_raw_train = ds_raw.take(round(ds_length*0.8)-1)
ds_raw_train_valid = ds_raw.take(round(ds_length*0.2)-1)

## Create dictionary

In [6]:
from collections import Counter

tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()

for example in ds_raw_train:
    for i in range(0,2):
        tokens = tokenizer.tokenize(example[0].numpy()[i])
        token_counts.update(tokens)

print("Größe des Wörterbuches: ", len(token_counts))

Größe des Wörterbuches:  99353


## Create a encoder and map the arrays

In [7]:
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
encoder.encode('This is a example')

def encode(text_tensor, label):
    #print(text_tensor)
    text0 = text_tensor.numpy()[0]
    text1 = text_tensor.numpy()[1]
    return encoder.encode(text0), encoder.encode(text1), label

def encode_map_fn(text, label):
    return tf.py_function(encode,
                          inp=[text, label],
                          Tout=[tf.int64, tf.int64, tf.int64])

ds_train = ds_raw_train.map(encode_map_fn)
ds_train_valid = ds_raw_train_valid.map(encode_map_fn)
for x in ds_train.take(5):
    print(x[0].shape)

(14,)
(10,)
(14,)
(11,)
(13,)


In [8]:
## Take a small subset
ds_subset = ds_train.take(8)
for example in ds_subset:
    print('Individual Shape:', example[0].shape)

## batching the datasets
ds_batched = ds_subset.padded_batch(
    4, padded_shapes=([-1], [-1], []))

for batch in ds_batched:
    print('Batch Shape:', batch[0].shape)
    print(batch)

Individual Shape: (14,)
Individual Shape: (10,)
Individual Shape: (14,)
Individual Shape: (11,)
Individual Shape: (13,)
Individual Shape: (17,)
Individual Shape: (4,)
Individual Shape: (7,)
Batch Shape: (4, 14)
(<tf.Tensor: shape=(4, 14), dtype=int64, numpy=
array([[ 1,  2,  3,  4,  5,  4,  6,  7,  8,  9, 10, 11,  9, 12],
       [ 1,  2,  3, 13, 14, 15, 16, 17, 18, 19,  0,  0,  0,  0],
       [28, 29, 30, 31,  3, 32, 14, 33, 34, 35, 36, 37, 38, 39],
       [46, 47, 30, 48, 49, 50, 28, 29, 30, 51, 52,  0,  0,  0]])>, <tf.Tensor: shape=(4, 15), dtype=int64, numpy=
array([[ 1,  2,  3,  4,  5,  4,  6,  7,  8,  9, 10, 11,  0,  0,  0],
       [ 1, 20, 21, 22,  3, 23, 24, 25,  3, 15, 16, 17, 18, 26, 27],
       [28, 29, 40, 32, 41, 42,  5, 43, 44, 45,  0,  0,  0,  0,  0],
       [53,  3, 54, 55, 56, 57, 58, 56,  2, 59,  5, 58, 57,  0,  0]])>, <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 0])>)
Batch Shape: (4, 17)
(<tf.Tensor: shape=(4, 17), dtype=int64, numpy=
array([[ 60,  61, 

# Batch the data


In [9]:
train_data = ds_train.padded_batch( 32, padded_shapes=([-1],[-1],[]))

valid_data = ds_train_valid.padded_batch( 32, padded_shapes=([-1],[-1],[]))

#test_data  = ds_test.padded_batch ( 32, padded_shapes=([-1],[-1],[]))

In [14]:
ds_subset = train_data.take(8)
for example in ds_subset:
    print('Individual Shape:', example)


Individual Shape: (<tf.Tensor: shape=(32, 29), dtype=int64, numpy=
array([[  1,   2,   3,   4,   5,   4,   6,   7,   8,   9,  10,  11,   9,
         12,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  1,   2,   3,  13,  14,  15,  16,  17,  18,  19,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 28,  29,  30,  31,   3,  32,  14,  33,  34,  35,  36,  37,  38,
         39,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 46,  47,  30,  48,  49,  50,  28,  29,  30,  51,  52,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 60,  61,  62,   9,  63,  64,  65,  66,  67,  68,  69,  70,  71,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 74,  30,  47,  38,  75,  76,  77,  78,  68,  79,  80,  81,  82,
         8

# Add the trainings-modell

In [11]:
from keras.layers import Embedding, SimpleRNN, Dense

lstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = False))

size_dic = len(token_counts) + 2

emb = Embedding(size_dic, 100, input_length=300)

input1 = tf.keras.Input(shape=(300,))
input2 = tf.keras.Input(shape=(300,))

e1 = emb(input1)
e2 = emb(input2)
x1 = lstm_layer(e1)
x2 = lstm_layer(e2)

mhd = lambda x: tf.keras.backend.abs(x[0] - x[1])
merged = tf.keras.layers.Lambda(function=mhd, output_shape= lambda x: x[0])([x1, x2])
preds = tf.keras.layers.Dense(1)(merged)
model = tf.keras.Model(inputs=[input1, input2], outputs=preds)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 300, 100)     9935500     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 128)          84480       ['embedding[0][0]',          

In [12]:
'''from keras.layers import Embedding, SimpleRNN, Dense

size_dic = len(token_counts) + 2
print("Größe Wörterbuch + 2", size_dic)

lstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))

size_dic = len(token_counts) + 2
print("Größe Wörterbuch + 2", size_dic)

model = tf.keras.Sequential()
model.add(Embedding(size_dic, output_dim = 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()'''

'from keras.layers import Embedding, SimpleRNN, Dense\n\nsize_dic = len(token_counts) + 2\nprint("Größe Wörterbuch + 2", size_dic)\n\nlstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))\n\nsize_dic = len(token_counts) + 2\nprint("Größe Wörterbuch + 2", size_dic)\n\nmodel = tf.keras.Sequential()\nmodel.add(Embedding(size_dic, output_dim = 32))\nmodel.add(SimpleRNN(32, return_sequences=True))\nmodel.add(SimpleRNN(32))\nmodel.add(Dense(1))\nmodel.summary()'

# Split tensors into two inputs

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])
history = model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5


ValueError: in user code:

    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/maxh/Gruppe_G_Fragen/venv/lib/python3.9/site-packages/keras/engine/input_

In [11]:
import os
model = tf.keras.models.load_model('./models/rnn.h5')
test_results = model.evaluate(train_data)
print(f'Test acc : {test_results}')
if not os.path.exists('models'):
    os.mkdir('models')

#model.save('models/rnn.h5')

10108/10108 [==============================] - 261s 26ms/step - loss: -4746.2764 - accuracy: 0.0130
Test acc : [-4746.2763671875, 0.013030407018959522]
